In [9]:
from bs4 import BeautifulSoup
import requests
import time
import re
import sqlite3
from selenium import webdriver
import json

In [157]:
driver = webdriver.Chrome()

In [320]:
driver.get('http://ticket.interpark.com/')

로그인
--------------------

In [11]:
%%writefile interpark.json


Overwriting interpark.json


In [159]:
with open('interpark.json','r') as  fp:
    account = json.load(fp)

In [160]:
session = requests.Session()

In [46]:
len([cookie['name'] for cookie in driver.get_cookies()]) # 로그인 전 쿠키 확인

45

In [37]:
def login(account, method='login'):
    try:
        login = driver.find_element_by_css_selector('#logstatus > a') # 로그인 버튼 클릭
        if method == 'login':
            if login and login.get_attribute('id') == 'aLogin':
                driver.get(login.get_attribute('href'))
                driver.find_element_by_id('UID').send_keys(account['id']) # id 입력
                driver.find_element_by_id('PWD').send_keys(account['pw']) # pw 입력
                driver.find_element_by_css_selector('.loginBtn .btnRed').click()
        elif method == 'logout':
            login.click()
    except:
        return False
    return True

In [161]:
login(account, method='login')

True

In [44]:
len([cookie['name'] for cookie in driver.get_cookies()]) # 로그인 후 쿠키 확인

51

In [51]:
## 세션 값만 보고 login, out 결정한다.
if 'ISession%5FID' not in [cookie['name'] for cookie in driver.get_cookies()]:
    login(account)
else:
    login(None, 'logout')

공연 검색
-----------------------

In [72]:
def gnbbtns(category='콘서트'):
    links = [_ for _ in driver.find_elements_by_css_selector('ul.gnbBtns a[class^=btn]')]
    titles = [_.find_element_by_css_selector('img').get_attribute('alt') for _ in links]
    
    if category in titles:
        return links[titles.index(category)].get_attribute('href')
    return False

In [321]:
url = gnbbtns('콘서트') 
if url:
    driver.get(url) # 해당 카테고리 페이지로 이동

In [90]:
## 공연 리스트 가져오기 
def hotIssue(issue=None):
    issues = [_ for _ in driver.find_elements_by_css_selector('.issue .list > dt.issue_obj > a')]
    title = [_.find_element_by_css_selector('span.txt1').text for _ in issues]
    
    if issue in title:
        return issues[title.index(issue)].get_attribute('href') # 찾는 공연 주소 return
    else:
        return issues[0].get_attribute('href')  # 찾는 공연 없으면 제일 앞에꺼 return

In [322]:
driver.get(hotIssue('박정현'))

In [208]:
def bookWindow(options=None):
    if options:
        calenderFrame = driver.find_element_by_css_selector('#ifrCalendar')
        if calenderFrame:
            driver.switch_to.frame(calenderFrame)
            ables = [_ for _ in driver.find_elements_by_css_selector('.cal_Table .able > a')]
            title = [_.text for _ in ables]

        if options in title:
            driver.execute_script(ables[title.index(options)].get_attribute('onclick'))
        else:
            print(ables[title.index(options)].text)
            driver.execute_script(ables[0].get_attribute('onclick'))


        driver.switch_to_default_content()
        
    driver.find_element_by_class_name('btn_rev').click()
    driver.switch_to.window(driver.window_handles[-1])

['2', '3', '4', '9', '10', '11']
fnSelectPlayDate(4, '20190810')


C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use driver.switch_to.default_content instead


In [168]:
def safeClose():
    if len(driver.window_handles) > 1:
        for _ in driver.window_handles[1:]:
            driver.switch_to.window(_)
            driver.close()
    driver.switch_to_window(driver.window_handles[0])
    driver.switch_to_default_content()

In [395]:
#### 일정 선택
bookWindow('11')
driver.execute_script("fnNextStep('P')")


############# 좌석 선택 ##############
driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmSeat'))
driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmSeatDetail'))
# position = [_.get_attribute('style') for _ in driver.find_elements_by_css_selector('.stySeat')]
# [_.get_attribute('onclick') for _ in driver.find_elements_by_css_selector('.stySeat')]
# [__ for __ in position]
selects = [_.get_attribute('onclick') for _ in driver.find_elements_by_css_selector('.stySeat')][:2] # 2개 선택
for _ in selects:
    driver.execute_script(_) 
    
###  다음 단계로 넘어감
driver.switch_to_default_content()
driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmSeat'))
driver.execute_script(driver.find_element_by_css_selector('.btnWrap > a').get_attribute('onclick')) 


############ 가격 / 할인선택 ###############

driver.switch_to_default_content()
driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmBookStep'))
driver.find_element_by_css_selector('.taL > select > option[value="2"]').click() ## 매수 선택
driver.switch_to_default_content()
driver.find_element_by_id('SmallNextBtnLink').click() # 다음 단계로 넘어감


########## 배송선택 / 주문자확인 ##################

driver.switch_to_default_content()
driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmBookStep'))
# driver.find_element_by_css_selector('#Delivery_24001 #Delivery').click()
driver.find_element_by_id('YYMMDD').send_keys('951118')
# driver.find_element_by_id('chkSyncAddress').click()

driver.switch_to_default_content()
driver.find_element_by_id('SmallNextBtnLink').click() # 다음 단계로 넘어감

driver.switch_to_default_content()
driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmBookStep'))
driver.find_element_by_css_selector('#Payment_22004 .chk').click()
driver.find_element_by_css_selector('#BankCode option[value="38051"]').click() ## 은행 선택
driver.switch_to_default_content()
driver.find_element_by_id('SmallNextBtnLink').click() # 다음 단계로 넘어감

# driver.switch_to_default_content()
# driver.switch_to_frame(driver.find_element_by_css_selector('#ifrmBookStep'))
# driver.find_element_by_css_selector('#CancelAgree').click()
# driver.find_element_by_css_selector('#CancelAgree2').click()
# driver.switch_to_default_content()
# driver.find_element_by_id('LargeNextBtnLink').click() # 다음 단계로 넘어감

['2', '3', '4', '9', '10', '11']
fnSelectPlayDate(5, '20190811')


C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use driver.switch_to.default_content instead
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use driver.switch_to.frame instead
  import sys
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use driver.switch_to.default_content instead
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use driver.switch_to.frame instead
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: use driver.switch_to.default_content instead
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: use driver.switch_to.frame instead
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning:

In [394]:
safeClose()
driver.switch_to_window(driver.window_handles[0])
driver.refresh()

C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use driver.switch_to.default_content instead
  import sys
C:\Users\skarn\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.window instead
  


선생님 좌석선택 알고리즘
-------------------